In [10]:
import pandas as pd
import tqdm
import requests
from datasets import Dataset
import os
import getpass
from clearml import Task

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

In [40]:
os.environ["OPENAI_API_KEY"] = getpass.getpass('Enter your key: ')

In [12]:
# clear ml credentials

In [13]:
test_data_path = "/Users/ekaterina.baru/projects/SpicyRAG/data/eval_dataset.xlsx"

In [14]:
# project:
#   name: "SpicyRag"
# 
# database:
#   user: "user"
#   password: "password"
#   name: "spicyragdb"
#   host: "localhost"
#   port: 5432
# 
# logging:
#   level: "INFO"
# 
# server:
#   host: "0.0.0.0"
#   port: 8000
# 
# llm:
#   model: "llama3.1-405b"
#   role: "user"
#   system_prompt: |
#     Вы являетесь виртуальным юридическим консультантом, специализирующимся на анализе и интерпретации юридических документов и правовых норм.
#     Ваша задача — помочь пользователю, используя предоставленный контекст, чтобы дать точный и обоснованный ответ на его юридический вопрос.
#     Контекст может содержать выдержки из законов, нормативных актов, судебных решений, договоров или других юридических текстов.
# 
#     Инструкции:
#       1. Внимательно проанализируй контекст и определите, какие части контекста наиболее релевантны, приводи всю полезную информацию из контекста
#       2. Используй релевантные фрагменты контекста (не пиши к чему обращаешься, не упоминай пунктов) для создания полного и юридически обоснованного ответа. Приводи цитаты из контекста в соответствии
#       3. Если контекст не совсем соответствует вопросу, пиши: "Ответ на данный вопрос отсутствует в контексте.". Но попробуй достать из контекста самую релевентную информацию для запрса
# 
#     ВАЖНО: не упоминай, что ты работаешь с контекстом! Отвечай структурно и по пунктам. Отвечай на русском языке
#   temperature: 0.15
#   top_p: 0.7
#   max_tokens: 4096
# 
# llm_rewriter:
#   model: "llama3.1-405b"
#   role: "user"
#   system_prompt: |
#     Вы - полезный помощник, который генерирует несколько поисковых запросов на основе одного входного запроса.
#     Выполните расширение запроса. Если существует несколько распространенных способов формулировки пользовательского вопроса или общих синонимов для ключевых слов в вопросе, обязательно верните несколько версий запроса с разными формулировками.
#     Если в тексте есть сокращения или слова, которые вам незнакомы, не пытайтесь их перефразировать.
#     НЕ ПИШИТЕ В ПЕРЕПИСАННЫХ ЗАПРОСАХ ОТВЕТ НА ЗАПРОС!
#     Напишите только 3 похожих запроса, вам не надо отвечать на запрос, не пытайтесь объяснять суть вопроса
#     В конце запроса ставь вопросительный знак
#     Не отвечайте на вопрос, не пиши ничего кроме 3 запросов
#     Для разделителя используйте \n
#   temperature: 0.2
#   top_p: 0.7
#   max_tokens: 1024
# 
# embedding_model:
#   batch_size: 16
#   model_name: "sentence-transformers/paraphrase-MiniLM-L6-v2"
#   model_type: ""
#   dimension: 384
#   prefix_query: "{}"
#   prefix_document: "{}"
# 
# elastic_params:
#   host: "elasticsearch"
#   port: 9200
#   index_name: "chunks"
# 
# retrieval:
#   vector_search_enabled: true
#   similarity_threshold: 0.7
#   top_k_vector: 3
#   fulltext_search_enabled: true
#   top_k_fulltext: 3
#   top_k : 5
# 
# data_sources:
#   excel_file: "./data/v2_ragas_npa_dataset_firstPart.xlsx"
#   text_file: "./data/hmao_npa.txt"
#   text_separator: "\n\n"
# 
# data_processing:
#   chunker:
#     py_class: interface.chunker.RecursiveCharacterTextSplitterChunker
#     kwargs:
#       chunk_size: 2048
#       chunk_overlap: 256
#       separators:
#         - '\n#{1,6} '
#         - '```\n'
#         - '\n\\*\\*\\*+\n'
#         - '\n---+\n'
#         - '\n___+\n'
#         - '\n\n'
#         - '\n'
#         - ' '
#         - ''
#         - ','
#         - '|'


In [34]:
def get_system_responses(questions):
    """
    Send requests to the /ask endpoint and collect responses.
    """
    responses = []
    for question in tqdm.tqdm(questions.itertuples()):
        try:
            response = requests.post("http://localhost:8000/ask/", json={"question": question[2]})
            response.raise_for_status()
            response_data = response.json()
            print(response_data["response"])
            responses.append(
                {
                    "question": question[2],
                    "response": response_data["response"],
                    "ground_truth": question[4],
                    "context": response_data["context"], 
                }
            )
        except Exception as e:
            print(f"Error while sending query: {question[1]}, error message: {e}")
            continue
    return responses

In [35]:
test_data = pd.read_excel(test_data_path)

In [36]:
system_responses = get_system_responses(test_data)

1it [00:06,  6.75s/it]

Конкурс среди оленеводов в ХМАО проводится с целью совершенствования профессионального мастерства и повышения престижа профессии оленевода. Кроме того, задачами конкурса являются выявление, пропаганда и внедрение в северном оленеводстве передовых приемов и методов труда, распространение опыта работы лучших оленеводов и привлечение молодежи


2it [00:12,  6.33s/it]

Согласно контексту, для экономии энергии в ХМАО-Югре можно модернизировать электросети и оборудование. В частности, предлагается:

* Увеличить выработку электрической энергии в комбинированном режиме
* Снизить бюджетные затраты на энергообеспечение удаленных территорий
* Модернизировать электросетевое


3it [00:18,  5.92s/it]

...

Для работников добровольной пожарной охраны Ханты-Мансийского автономного округа - Югры предусмотрены следующие меры социальной защиты:

1. Оплата дополнительного отпуска, предоставляемого добровольным пожарным на время участия в тушении пожаров, несения службы (дежурства) в расположении добровольной пожарной команды или добровольной пожарной дружины либо прохождения


119it [11:14,  5.77s/it]

Ответ на данный вопрос отсутствует в контексте. Однако, в контексте 3 упоминается, что контроль за деятельностью заказника осуществляется исполнительным органом государственной власти автономного округа, осуществляющим функции по организации и функционированию особо охраняемых природных территорий регионального значения, а также иными государственными органами и учреждениями в пределах их компетенции. Это может подразумевать, что мер


120it [11:19,  5.66s/it]

В бюджете Управления внутренних дел автономного округа на приобретение плавсредств и снегоходов для подразделений аппарата Управления внутренних дел автономного округа в 2008 году были предусмотрены расходы в размере 1900 единиц.


In [41]:
data_samples = {
    "question": [response["question"] for response in system_responses],
    "answer": [response["response"] for response in system_responses],
    "contexts": [response["context"] for response in system_responses],
    "ground_truth": [response["ground_truth"] for response in system_responses],
}
dataset = Dataset.from_dict(data_samples)

In [43]:
result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating:   0%|          | 0/476 [00:00<?, ?it/s]

Exception raised in Job[75]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-UZZpjHJPzrDNOTXMz3GCGmjK on tokens per min (TPM): Limit 200000, Used 198860, Requested 14831. Please try again in 4.107s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[27]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-UZZpjHJPzrDNOTXMz3GCGmjK on tokens per min (TPM): Limit 200000, Used 196034, Requested 9374. Please try again in 1.622s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[1]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-UZZpjHJPzrDNOTXMz3GCGmjK on tokens per min (TPM): Limit 200000, 

{'context_precision': 1.0000, 'faithfulness': 0.6750, 'answer_relevancy': 0.5068, 'context_recall': 0.8589}

In [31]:
# {'context_precision': 0.8770, 'faithfulness': 0.5064, 'answer_relevancy': 0.4106, 'context_recall': 0.9133}

In [44]:
import json

with open("exp_eval.json", 'w') as f:
    json.dump(system_responses, f, indent=4) 